<a href="https://colab.research.google.com/github/Nilanshrajput/AdvitiyaHackathon/blob/master/BertV2_Final_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Installing hugging face library for Fine tuning Bert Model
!pip install transformers

     |████████████████████████████████| 481kB 3.5MB/s 
     |████████████████████████████████| 1.0MB 65.4MB/s 
     |████████████████████████████████| 870kB 60.4MB/s 
     |████████████████████████████████| 3.1MB 48.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=1eaf2390782ced39936722d441525b542e7eadc5a5cdca9cb43e2dcbf80b6fc5
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [1]:
import torch
import numpy as np
import pandas as pd
from transformers import BertModel, BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from torch import nn

from time import time




In [0]:

%%capture
from tqdm import tnrange
from tqdm import tqdm_notebook as tqdm

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
drive_pt="/content/drive/My Drive/dl_projects/AdvitiyaHackathon/"

git_path = "AdvitiyaHackathon/"
path = drive_pt+"ScrappeData"

In [0]:
bert_model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [5]:

from keras.preprocessing.sequence import pad_sequences

#DataSet class
class LoadDataset(Dataset):

    def __init__(self, filename, maxlen):

        # Store the contents of the file in a pandas dataframe
        self.df = pd.read_csv(filename, delimiter=',',index_col=0)

        # Initialize the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        # Define the Maxlength for padding/truncating
        self.maxlen = maxlen

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        # Selecting the sentence and label at the specified index in the data frame
        sentence = self.df.loc[index, 'text']
        sentence = sentence[10:]
        #sentence = re.sub(' +', ' ',sentence)
        label = torch.tensor((self.df.loc[index, 'cat']))

        # Tokenize the sentence
        tokens_ids = self.tokenizer.encode(str(sentence),add_special_tokens = True,max_length = self.maxlen, pad_to_max_length=True)
        """
        tokens_ids = pad_sequences([tokens_ids], maxlen=self.maxlen, dtype="long", 
                          value=0, truncating="post", padding="post")
        """   
        # Inserting the CLS and SEP token at the beginning and end of the sentence
        #tokens = ['[CLS]'] + tokens 
        """
        if len(tokens_ids) < self.maxlen:
            tokens_ids = [tokens_ids + [0] for _ in range(self.maxlen - len(tokens_ids))]
        else:
            tokens_ids = tokens_ids[:self.maxlen-1]         
        """  
        # Padding/truncating the sentences to the maximum length


        # Convert the sequence to ids with BERT Vocabulary
        #tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        
        # Converting the list to a pytorch tensor
        tokens_ids_tensor = torch.tensor(tokens_ids)

        # Obtaining the attention mask
        attn_mask = (tokens_ids_tensor != 0).long()

        return tokens_ids_tensor, attn_mask, label

Using TensorFlow backend.


In [0]:
# Creating instances of training and validation set
train_set = LoadDataset(filename = drive_pt+'train/all_text_train_all.csv', maxlen = 200)
val_set = LoadDataset(filename = drive_pt+'train/val_all.csv', maxlen = 200)


In [0]:

# Creating intsances of training and validation dataloaders
train_loader = DataLoader(train_set, batch_size = 16, shuffle = True, num_workers = 5)
val_loader = DataLoader(val_set, batch_size = 32, shuffle = True, num_workers = 5)

In [8]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 4, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = True, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

#model = PatentClassifier(freeze_bert= False)

# Tell pytorch to run this model on the GPU.
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [9]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [0]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 4e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


In [0]:

from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 30

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_loader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [0]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:

import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [14]:


import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [15]:
#Loading our model
import os
save_path = drive_pt+ "Models/"


"""# Loading the checkpoints for resuming training
checkpoint = torch.load(save_path+'finalbertv4_claims_512_8_20.pth')
model.load_state_dict(checkpoint['model_state_dict'])"""

"# Loading the checkpoints for resuming training\ncheckpoint = torch.load(save_path+'finalbertv4_claims_512_8_20.pth')\nmodel.load_state_dict(checkpoint['model_state_dict'])"

In [16]:
import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_loader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_loader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
       
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_loader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in val_loader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 30 ========
Training...
  Batch    40  of    225.    Elapsed: 0:00:33.
  Batch    80  of    225.    Elapsed: 0:01:03.
  Batch   120  of    225.    Elapsed: 0:01:32.
  Batch   160  of    225.    Elapsed: 0:02:02.
  Batch   200  of    225.    Elapsed: 0:02:32.

  Average training loss: 0.32
  Training epcoh took: 0:02:50

Running Validation...
  Accuracy: 0.99
  Validation took: 0:00:08

======== Epoch 2 / 30 ========
Training...
  Batch    40  of    225.    Elapsed: 0:00:34.
  Batch    80  of    225.    Elapsed: 0:01:04.
  Batch   120  of    225.    Elapsed: 0:01:33.
  Batch   160  of    225.    Elapsed: 0:02:03.
  Batch   200  of    225.    Elapsed: 0:02:32.

  Average training loss: 0.07
  Training epcoh took: 0:02:51

Running Validation...
  Accuracy: 1.00
  Validation took: 0:00:08

======== Epoch 3 / 30 ========
Training...
  Batch    40  of    225.    Elapsed: 0:00:33.
  Batch    80  of    225.    Elapsed: 0:01:03.
  Batch   120  of    225.    Elapsed: 0:01:32.

In [0]:
# Saving our model
import os
save_path = drive_pt+ "Models/"
if not os.path.isdir(save_path):
    os.mkdir(save_path)

torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, save_path+'all_final_200_32_30.pth')

Evaluation

In [0]:
# Saving our model
import os
save_path = drive_pt+ "Models/"
model1,model2,model3,model4,model5, model6 = model,model,model,model,model,model

# Loading the checkpoints for resuming training
checkpoint = torch.load(save_path+'finalbertv5_abstract_512_8_20.pth')
model1.load_state_dict(checkpoint['model_state_dict'])

# Loading the checkpoints for resuming training
checkpoint = torch.load(save_path+'bertv6_classification_512_8_20.pth')
model2.load_state_dict(checkpoint['model_state_dict'])

# Loading the checkpoints for resuming training
checkpoint = torch.load(save_path+'finalbertv4_claims_512_8_20.pth')
#model3.load_state_dict(checkpoint['model_state_dict'])

# Loading the checkpoints for resuming training
checkpoint = torch.load(save_path+'all_final_calims_128_32_30.pth')
#model4.load_state_dict(checkpoint['model_state_dict'])

# Loading the checkpoints for resuming training
checkpoint = torch.load(save_path+'all_final_calims_300_16_20.pth')
#model5.load_state_dict(checkpoint['model_state_dict'])

# Loading the checkpoints for resuming training
checkpoint = torch.load(save_path+'all_final_200_32_30.pth')
#model6.load_state_dict(checkpoint['model_state_dict'])



In [0]:
#auxilary  Functions for Evaluating on Test set
def most_frequent(List): 
    counter = 0
    num = List[0] 
      
    for i in List: 
        curr_frequency = List.count(i) 
        if(curr_frequency> counter): 
            counter = curr_frequency 
            num = i 
  
    return num 



def return_model_output(model, sentences, dev ):
  results=[]
  model.eval()
  for sentence in sentences:
    b_input_ids, b_input_mask = pre_process(sentence, maxlen = 512,device=dev)
    output = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
    logits = output[0]
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    cat = np.argmax(logits, axis=1).flatten()
    results.append(int(cat))

  return results

def pre_process(sentence, maxlen,device):
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  #sentence = sentence[50:]
  tokens_ids = tokenizer.encode(str(sentence),add_special_tokens = True,max_length = maxlen, pad_to_max_length=True)
  # Converting the list to a pytorch tensor
  tokens_ids_tensor = torch.tensor(tokens_ids)

  # Obtaining the attention mask
  attn_mask = (tokens_ids_tensor != 0).long()

  return tokens_ids_tensor.unsqueeze(dim = 0).to(device), attn_mask.unsqueeze(dim = 0).to(device)


In [0]:
#Loading Test set
df1 = pd.read_csv(drive_pt+"claims_final_test_clean.csv", index_col= 0 )
df2 = pd.read_csv(drive_pt+"classification_final_test_clean.csv", index_col= 0 )
df3 = pd.read_csv(drive_pt+"abstract_final_test_clean.csv", index_col= 0 )

In [29]:
#Predicting on Test Set

for index in tnrange(0,len(df1)):
  
  sentence1 = df1.loc[index, 'text']
  sentence2 = df2.loc[index, 'text']
  sentence3 = df3.loc[index, 'text']
  sentences = [sentence1, sentence2, sentence3]
  results = return_model_output(model1, sentences= sentences,dev = device)
  #results += return_model_output(model2, sentences= sentences,dev = device)
  results += return_model_output(model3, sentences= sentences,dev = device)
  results += return_model_output(model4, sentences= sentences,dev = device)
  results += return_model_output(model5, sentences= sentences,dev = device)
  results += return_model_output(model6, sentences= sentences,dev = device)

  final_cat = most_frequent(results)
  df1.loc[index, 'cat'] = final_cat


In [0]:
#get the test data 
dftest = pd.read_csv(drive_pt+"test.csv")

In [0]:
dftest["Category"] = df1.iloc[:,1]

In [0]:
#Cnpnverting encoded cateegoris to textual form

names = ["Non-Alcohol","Alcohol", "Non-Autonomous Vehicles", "Autonomous Vehicles"]
for i in range(0,4):
  mask = dftest.Category == i
  column_name = 'Category'
  dftest.loc[mask, column_name] = names[i]

In [33]:
#checking from predicted
dftest.loc[34, "Category"]

'Alcohol'

In [0]:
#saving csv
dftest.to_csv("submission_5lastmodel.csv")